In [ ]:
from bs4 import BeautifulSoup
from pandas import json_normalize
import requests
import feedparser
import os

In [ ]:
rss_links = {
    "thoi_su": ["https://thanhnien.vn/rss/thoi-su.rss", "https://danviet.vn/rss/tin-tuc-1001.rss", "https://tienphong.vn/rss/thoi-su-421.rss", "https://vtc.vn/rss/thoi-su.rss", "https://nld.com.vn/rss/thoi-su.rss", "https://thanhnien.vn/rss/thoi-su/chinh-tri.rss", "https://thanhnien.vn/rss/thoi-su/phap-luat.rss", "https://thanhnien.vn/rss/thoi-su/dan-sinh.rss", "https://thanhnien.vn/rss/thoi-su/phong-su--dieu-tra.rss", "https://thanhnien.vn/rss/thoi-su/thoi-luan.rss", "https://danviet.vn/rss/kinh-da-trong-1059.rss", "https://danviet.vn/rss/chinh-tri-1002.rss", "https://danviet.vn/rss/xa-hoi-1003.rss", "https://danviet.vn/rss/phong-su-ghi-chep-1182.rss"],
    "the_gioi": ["https://thanhnien.vn/rss/the-gioi.rss", "https://thanhnien.vn/rss/the-gioi/goc-nhin.rss", "https://thanhnien.vn/rss/the-gioi/ho-so.rss", "https://thanhnien.vn/rss/the-gioi/chuyen-la.rss", "https://www.nguoiduatin.vn/rss/the-gioi.rss", "https://www.nguoiduatin.vn/rss/tieu-diem-the-gioi.rss", "https://nld.com.vn/rss/quoc-te.rss", "https://vtc.vn/rss/the-gioi.rss", "https://danviet.vn/rss/the-gioi-1007.rss", "https://tienphong.vn/rss/the-gioi-5.rss"],
    "suc_khoe": ["https://thanhnien.vn/rss/suc-khoe.rss", "https://thanhnien.vn/rss/suc-khoe/khoe-dep-moi-ngay.rss", "https://thanhnien.vn/rss/suc-khoe/lam-dep.rss", "https://www.nguoiduatin.vn/rss/suc-khoe.rss", "https://nld.com.vn/rss/suc-khoe.rss", "https://vtc.vn/rss/suc-khoe.rss", "https://danviet.vn/rss/suc-khoe-1191.rss", "https://tienphong.vn/rss/suc-khoe-210.rss"],
    "kinh_te": ["https://thanhnien.vn/rss/kinh-te.rss", "https://thanhnien.vn/rss/kinh-te/kinh-te-xanh.rss", "https://thanhnien.vn/rss/kinh-te/chinh-sach-phat-trien.rss", "https://thanhnien.vn/rss/kinh-te/ngan-hang.rss", "https://thanhnien.vn/rss/kinh-te/chung-khoan.rss", "https://thanhnien.vn/rss/kinh-te/doanh-nghiep.rss", "https://www.nguoiduatin.vn/rss/kinh-doanh.rss", "https://www.nguoiduatin.vn/rss/tai-chinh-ngan-hang.rss", "https://nld.com.vn/rss/kinh-te.rss", "https://vtc.vn/rss/kinh-te.rss", "https://danviet.vn/rss/kinh-te-1004.rss", "https://danviet.vn/rss/thi-truong-1069.rss", "https://danviet.vn/rss/kinh-te-so-1218.rss", "https://tienphong.vn/rss/kinh-te-3.rss"],
    "giao_duc": ["https://thanhnien.vn/rss/giao-duc.rss", "https://thanhnien.vn/rss/giao-duc/tuyen-sinh.rss", "https://thanhnien.vn/rss/giao-duc/chon-nghe-chon-truong.rss", "https://thanhnien.vn/rss/giao-duc/du-hoc.rss", "https://thanhnien.vn/rss/giao-duc/nha-truong.rss", "https://www.nguoiduatin.vn/rss/giao-duc.rss", "https://nld.com.vn/rss/giao-duc-khoa-hoc.rss", "https://vtc.vn/rss/giao-duc.rss", "https://danviet.vn/rss/giao-duc-1196.rss", "https://danviet.vn/rss/hoc-duong-1197.rss", "https://danviet.vn/rss/tuyen-sinh-1198.rss", "https://danviet.vn/rss/du-hoc-1199.rss", "https://tienphong.vn/rss/giao-duc-71.rss"],
    "du_lich": ["https://thanhnien.vn/rss/du-lich.rss", "https://thanhnien.vn/rss/du-lich/kham-pha.rss", "https://thanhnien.vn/rss/du-lich/tin-tuc-su-kien.rss", "https://nld.com.vn/rss/du-lich-xanh.rss", "https://danviet.vn/rss/du-lich-the-gioi-1112.rss", "https://danviet.vn/rss/du-lich-1097.rss", "https://danviet.vn/rss/mon-ngon-diem-dep-1099.rss", "https://tienphong.vn/rss/du-lich-220.rss"],
    "van_hoa": ["https://thanhnien.vn/rss/van-hoa.rss", "https://thanhnien.vn/rss/van-hoa/cau-chuyen-van-hoa.rss", "https://thanhnien.vn/rss/van-hoa/khao-cuu.rss", "https://www.nguoiduatin.vn/rss/van-hoa.rss", "https://vtc.vn/rss/van-hoa-giai-tri.rss", "https://danviet.vn/rss/doi-song-van-hoa-1188.rss", "https://danviet.vn/rss/van-hoa-giai-tri-1006.rss"],
    "the_thao": ["https://thanhnien.vn/rss/the-thao.rss", "https://thanhnien.vn/rss/the-thao/bong-da-viet-nam.rss", "https://thanhnien.vn/rss/the-thao/bong-da-quoc-te.rss", "https://thanhnien.vn/rss/the-thao/the-thao-cong-dong.rss", "https://thanhnien.vn/rss/the-thao/the-thao-khac.rss", "https://www.nguoiduatin.vn/rss/the-thao.rss", "https://nld.com.vn/rss/the-thao.rss", "https://vtc.vn/rss/the-thao.rss", "https://tienphong.vn/rss/the-thao-11.rss"],
    "cong_nghe": ["https://thanhnien.vn/rss/cong-nghe-game.rss", "https://thanhnien.vn/rss/cong-nghe-game/tin-tuc-cong-nghe.rss", "https://thanhnien.vn/rss/cong-nghe-game/blockchain.rss", "https://thanhnien.vn/rss/cong-nghe-game/san-pham.rss", "https://thanhnien.vn/rss/cong-nghe-game/xu-huong-chuyen-doi-so.rss", "https://www.nguoiduatin.vn/rss/cong-nghe.rss", "https://nld.com.vn/rss/cong-nghe.rss", "https://vtc.vn/rss/khoa-hoc-cong-nghe.rss", "https://danviet.vn/rss/cong-nghe-1030.rss", "https://danviet.vn/rss/danh-gia-san-pham-1032.rss", "https://danviet.vn/rss/dien-thoai-may-tinh-1033.rss", "https://tienphong.vn/rss/cong-nghe-khoa-hoc-46.rss"],
    "phap_luat": ["https://thanhnien.vn/rss/thoi-su/chinh-tri.rss", "https://thanhnien.vn/rss/thoi-su/phap-luat.rss", "https://www.nguoiduatin.vn/rss/phap-luat.rss", "https://www.nguoiduatin.vn/rss/ho-so-dieu-tra.rss", "https://nld.com.vn/rss/phap-luat.rss", "https://vtc.vn/rss/phap-luat.rss", "https://danviet.vn/rss/phap-luat-1008.rss", "https://danviet.vn/rss/phap-dinh-1067.rss", "https://tienphong.vn/rss/phap-luat-12.rss"]
}

In [ ]:
train_rss_urls = {
    "tuoi_tre": ["https://tuoitre.vn/rss/tin-moi-nhat.rss", "https://tuoitre.vn/rss/thoi-su.rss", "https://tuoitre.vn/rss/the-gioi.rss", "https://tuoitre.vn/rss/phap-luat.rss", "https://tuoitre.vn/rss/kinh-doanh.rss", "https://tuoitre.vn/rss/nhip-song-so.rss", "https://tuoitre.vn/rss/xe.rss", "https://tuoitre.vn/rss/nhip-song-tre.rss", "https://tuoitre.vn/rss/van-hoa.rss", "https://tuoitre.vn/rss/giai-tri.rss", "https://tuoitre.vn/rss/the-thao.rss", "https://tuoitre.vn/rss/giao-duc.rss", "https://tuoitre.vn/rss/khoa-hoc.rss", "https://tuoitre.vn/rss/suc-khoe.rss", "https://tuoitre.vn/rss/gia-that.rss", "https://tuoitre.vn/rss/thu-gian.rss", "https://tuoitre.vn/rss/ban-doc-lam-bao.rss", "https://tuoitre.vn/rss/du-lich.rss"],
    "thanh_nien": ["https://thanhnien.vn/rss/home.rss", "https://thanhnien.vn/rss/thoi-su.rss", "https://thanhnien.vn/rss/chao-ngay-moi.rss", "https://thanhnien.vn/rss/the-gioi.rss", "https://thanhnien.vn/rss/kinh-te.rss", "https://thanhnien.vn/rss/doi-song.rss", "https://thanhnien.vn/rss/suc-khoe.rss", "https://thanhnien.vn/rss/gioi-tre.rss", "https://thanhnien.vn/rss/tieu-dung-thong-minh.rss", "https://thanhnien.vn/rss/giao-duc.rss", "https://thanhnien.vn/rss/du-lich.rss", "https://thanhnien.vn/rss/van-hoa.rss", "https://thanhnien.vn/rss/giai-tri.rss", "https://thanhnien.vn/rss/the-thao.rss", "https://thanhnien.vn/rss/cong-nghe-game.rss", "https://thanhnien.vn/rss/xe.rss", "https://thanhnien.vn/rss/thoi-trang-tre.rss", "https://thanhnien.vn/rss/ban-doc.rss", "https://thanhnien.vn/rss/video.rss", "https://thanhnien.vn/rss/dien-dan.rss", "https://thanhnien.vn/rss/podcast.rss", "https://thanhnien.vn/rss/nhat-ky-tet-viet.rss", "https://thanhnien.vn/rss/magazine.rss", "https://thanhnien.vn/rss/cung-con-di-tiep-cuoc-doi.rss", "https://thanhnien.vn/rss/ban-can-biet.rss", "https://thanhnien.vn/rss/blog-phong-vien.rss", "https://thanhnien.vn/rss/toi-viet.rss", "https://thanhnien.vn/rss/tno.rss", "https://thanhnien.vn/rss/tin-24h.rss", "https://thanhnien.vn/rss/thi-truong.rss", "https://thanhnien.vn/rss/tin-nhanh-360.rss"],
    "nld": ["https://nld.com.vn/rss/home.rss", "https://nld.com.vn/rss/thoi-su.rss", "https://nld.com.vn/rss/quoc-te.rss", "https://nld.com.vn/rss/lao-dong.rss", "https://nld.com.vn/rss/ban-doc.rss", "https://nld.com.vn/rss/phap-luat.rss", "https://nld.com.vn/rss/van-hoa-van-nghe.rss", "https://nld.com.vn/rss/giai-tri.rss", "https://nld.com.vn/rss/cong-nghe.rss", "https://nld.com.vn/rss/du-lich-xanh.rss", "https://nld.com.vn/rss/chuyen-trang-phu-nu.rss", "https://nld.com.vn/rss/gia-dinh.rss", "https://nld.com.vn/rss/dia-oc.rss", "https://nld.com.vn/rss/khoa-hoc.rss", "https://nld.com.vn/rss/du-lich.rss", "https://nld.com.vn/rss/moi-truong.rss", "https://nld.com.vn/rss/hoi-nong-dap-nhanh.rss", "https://nld.com.vn/rss/truy-vet-mang-xa-hoi.rss", "https://nld.com.vn/rss/noi-thang.rss", "https://nld.com.vn/rss/goc-nhin.rss", "https://nld.com.vn/rss/trich-dan-nong.rss", "https://nld.com.vn/rss/vung-mien.rss", "https://nld.com.vn/rss/doc-quyen.rss", "https://nld.com.vn/rss/mai-vang.rss", "https://nld.com.vn/rss/doi-song.rss", "https://nld.com.vn/rss/tu-hao-co-to-quoc.rss", "https://nld.com.vn/rss/bien-dao.rss", "https://nld.com.vn/rss/ly-tuong-song.rss"],
}

test_rss_urls = {
    "nld": ["https://nld.com.vn/rss/kinh-te.rss", "https://nld.com.vn/rss/suc-khoe.rss"],
}

val_rss_urls = {
    "nld": ["https://nld.com.vn/rss/giao-duc-khoa-hoc.rss", "https://nld.com.vn/rss/the-thao.rss"],
}

In [ ]:
def get_news_content(url):
    result = ""
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find("div", "detail-content")
    if div is None:
        return ""
    texts = div.find_all("p")
    for text in texts:
        result += " " + text.get_text()
        
    return result

In [ ]:
def crawl_text(rss, path):
    if (not os.path.isdir(f"{path}")):
        os.mkdir(f"{path}")
        
    for article in rss:
        if (not os.path.isdir(f"{path}{article}")):
            os.mkdir(f"{path}{article}")
            
        print(article)

        index = 15501
        for rss_url in rss[article]:
            print(rss_url)

            news_feed = feedparser.parse(rss_url)
            df_news_feed = json_normalize(news_feed.entries)
            
            if "summary" in df_news_feed.keys():
                for text, news_url in zip(df_news_feed.summary, df_news_feed.link):
                    soup = BeautifulSoup(text, "html.parser")
                    print(news_url)
                    with open(os.path.join(f"{path}{article}", f'{index}.txt'), 'w', encoding='utf-8') as f:
                        try:
                            f.write(soup.get_text() + " " + get_news_content(news_url))
                        except:
                            continue
                        # f.write(soup.get_text())
                    index += 1

In [ ]:
crawl_text(train_rss_urls, "./temp/")
crawl_text(test_rss_urls, "./test_set/")
crawl_text(val_rss_urls, "./val_set/")